# Intersect catchment with ESA soil moisture project
# Finds the mean SM of each HRU in the model setup with rasterstats.

### Note
The rasterstats function `ZonalStatistics` automatically adds the calculated value to the shapefile used as input to the function. The workflow is thus:
1. Find the source catchment shapefile;
2. Copy the source catchment shapefile to the destintion location;
3. Run the zonal statistics algorithm on the copy.

In [3]:
# modules
import os
import sys
from pathlib import Path
from shutil import copyfile
from datetime import datetime
import geopandas as gpd
import rasterstats
import pandas as pd
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import regionmask
import dask
# Import libraries
import rioxarray as riox
from rasterio.enums import Resampling


#### Control file handling

In [4]:
# Easy access to control file folder
controlFolder = Path('../../0_control_files')

In [5]:
# Store the name of the 'active' file in a variable
controlFile = 'control_Yukon_Merit.txt'

In [6]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line and not line.startswith('#'):
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [7]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Find location of shapefile and DEM

In [8]:
# Catchment shapefile path & name
catchment_path = read_from_control(controlFolder/controlFile,'catchment_shp_path')
catchment_name = read_from_control(controlFolder/controlFile,'catchment_shp_name')

In [9]:
# Specify default path if needed
if catchment_path == 'default':
    catchment_path = make_default_path('shapefiles/catchment') # outputs a Path()
else:
    catchment_path = Path(catchment_path) # make sure a user-specified path is a Path()

In [10]:
# DEM path & name
mod10a1_path = read_from_control(controlFolder/controlFile,'observation_snow_mod10a1_path')
mod10a1_name = read_from_control(controlFolder/controlFile,'observation_snow_mod10a1_name')

In [11]:
# Specify default path if needed
if mod10a1_path == 'default':
    mod10a1_path = make_default_path('observations/MOD10A1/6_tif_multiband') # outputs a Path()
else:
    mod10a1_path = Path(mod10a1_path) # make sure a user-specified path is a Path()

mod10a1_path

PosixPath('/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/MOD10A1/6_tif_multiband')

#### Find where the intersection needs to go

In [12]:
# Intersected shapefile path and name
intersect_path = read_from_control(controlFolder/controlFile,'intersect_mod10a1_path')
intersect_name = read_from_control(controlFolder/controlFile,'intersect_mod10a1_name')
print(intersect_name)
print(intersect_path)

catchment_with_mod10a1.shp
default


In [13]:
# Specify default path if needed
if intersect_path == 'default':
    intersect_path = make_default_path('shapefiles/catchment_intersection/with_ESA_SM') # outputs a Path()
else:
    intersect_path = Path(intersect_path) # make sure a user-specified path is a Path()

In [14]:
# Make the folder if it doesn't exist
intersect_path.mkdir(parents=True, exist_ok=True)

#### Copy the source catchment shapefile into the destination location

In [15]:
# Find the name without extension
catchment_base = catchment_name.replace('.shp','')

In [16]:
# Loop over directory contents and copy files that match the filename of the shape
for file in os.listdir(catchment_path):
    if catchment_base in file: # copy only the relevant files in case there are more than 1 .shp files
        
        # make the output file name
        _,ext = os.path.splitext(file)                    # extension of current file
        basefile,_ = os.path.splitext(intersect_name)     # name of the intersection file w/o extension
        newfile = basefile + ext                          # new name + old extension
        
        # copy
        copyfile(catchment_path/file, intersect_path/newfile);

## Get the SUMMA output file

In [17]:
simulation_path = read_from_control(controlFolder/controlFile,'experiment_output_summa')
simulation_name = read_from_control(controlFolder/controlFile,'experiment_id')

In [18]:
# Specify default path if needed
if simulation_path == 'default':
    simulation_path = make_default_path('simulations/' + simulation_name + '/SUMMA/' + simulation_name + '_day.nc')
    simulation_path = Path(simulation_path) # make sure a user-specified path is a Path()

simulation_path

PosixPath('/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/simulations/run_Yukon_Merit_1/SUMMA/run_Yukon_Merit_1_day.nc')

## Rasterstats analysis

In [19]:
ds = xr.open_dataset(simulation_path)

In [20]:
ds

<xarray.Dataset> Size: 16GB
Dimensions:               (time: 15705, hru: 18225, gru: 18225)
Coordinates:
  * time                  (time) datetime64[ns] 126kB 1980-01-02 ... 2022-12-31
  * hru                   (hru) int64 146kB 81001460 81001703 ... 81036235
  * gru                   (gru) int64 146kB 81001460 81001703 ... 81036235
Data variables:
    scalarSWE             (time, hru) float64 2GB ...
    scalarAquiferStorage  (time, hru) float64 2GB ...
    scalarTotalSoilWat    (time, hru) float64 2GB ...
    scalarInfiltration    (time, hru) float64 2GB ...
    scalarTotalET         (time, hru) float64 2GB ...
    scalarTotalRunoff     (time, hru) float64 2GB ...
    scalarNetRadiation    (time, hru) float64 2GB ...
    hruId                 (hru) int64 146kB ...
    gruId                 (gru) int64 146kB ...

#### Spatial analysis

In [21]:
# Convert Path() to string for QGIS
catchment_file = str(intersect_path/intersect_name) # needs to be the coped file because output is automatically added to this
mod10a1_file = str(mod10a1_path/mod10a1_name)

In [22]:
layer_polygon = catchment_file
layer_SM = '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/dap.ceda.ac.uk/neodc/esacci/soil_moisture/data/daily_files/ACTIVE/v08.1/2005/ESACCI-SOILMOISTURE-L3S-SSMS-ACTIVE-20051201000000-fv08.1.nc'


print(layer_SM)
print(layer_polygon)

/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/dap.ceda.ac.uk/neodc/esacci/soil_moisture/data/daily_files/ACTIVE/v08.1/2005/ESACCI-SOILMOISTURE-L3S-SSMS-ACTIVE-20051201000000-fv08.1.nc
/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/shapefiles/catchment_intersection/with_ESA_SM/catchment_with_mod10a1.shp


In [23]:
bbox = read_from_control(controlFolder/controlFile,'forcing_raw_space').split('/')
lat_max = bbox[0]
lon_min = bbox[1]
lat_min = bbox[2]
lon_max = bbox[3]

In [24]:
raster_file = rio.open(total_path)
raster = raster_file.read(1)
raster_file.rio.width


NameError: name 'total_path' is not defined

In [25]:

 
# Read raster 

raster = riox.open_rasterio(total_path, crs = 'EPSG:4326')
raster = raster.rio.write_crs('EPSG:4326')
upscale_factor = 100
 
# Caluculate new height and width using upscale_factor
new_width = raster.rio.width * upscale_factor
new_height = raster.rio.height * upscale_factor
 
#upsample raster
up_sampled = raster.rio.reproject(raster.rio.crs, shape=(int(new_height), int(new_width)), resampling=Resampling.bilinear)
 


print(raster.rio.resolution(), up_sampled.rio.resolution())
# ((500.0, -500.0), (250.0, -250.0))
 
print(raster.shape, up_sampled.shape)
# ((1, 2660, 2305), (1, 5320, 4610))


NameError: name 'total_path' is not defined

In [32]:
files = []
for year in range(1992,2023):
    year_dir_path = '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/dap.ceda.ac.uk/neodc/esacci/soil_moisture/data/daily_files/ACTIVE/v08.1/' + str(year) + '/'

    print('Starting on year: ' + str(year))
    raster_path = year_dir_path + 'ESA_SM_' + str(year) + '.tif'
    #if not os.path.isfile(raster_path):

    dsYear = ds.sel(time = slice(str(year) + '-01-01', str(year) + '-12-31'))

    year_files = []
    for file in os.listdir(year_dir_path):
        if file.endswith('.nc'):
            files.append(Path(year_dir_path + file))
            year_files.append(Path(year_dir_path + file))

    dsSM = xr.open_mfdataset(year_files)
    dsSMslice = dsSM.sel(lat = slice(lat_max,lat_min), lon = slice(lon_max, lon_min))
    dsSMslice.sm.rio.to_raster(raster_path)
    #else:
    #    print('file already exists, skipping: ' + raster_path)


#dsSM = xr.open_mfdataset(files)
#dsSM = dsSM.sel(lat = slice(lat_max,lat_min), lon = slice(lon_max, lon_min))
#total_path = '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/2_merged_geotiff/ESA_SM.tif'
#dsSM.sm.rio.to_raster(total_path)


Starting on year: 1992
Starting on year: 1993
Starting on year: 1994
Starting on year: 1995
Starting on year: 1996
Starting on year: 1997
Starting on year: 1998
Starting on year: 1999
Starting on year: 2000
Starting on year: 2001
Starting on year: 2002
Starting on year: 2003
Starting on year: 2004
Starting on year: 2005
Starting on year: 2006
Starting on year: 2007
Starting on year: 2008
Starting on year: 2009
Starting on year: 2010
Starting on year: 2011
Starting on year: 2012
Starting on year: 2013
Starting on year: 2014
Starting on year: 2015
Starting on year: 2016
Starting on year: 2017
Starting on year: 2018
Starting on year: 2019
Starting on year: 2020
Starting on year: 2021
Starting on year: 2022


In [27]:
# Read raster 
raster = riox.open_rasterio(raster_path)#, crs = 'EPSG:4326')
raster = raster.rio.write_crs('EPSG:4326')
upscale_factor = 16
 
# Caluculate new height and width using upscale_factor
new_width = raster.rio.width * upscale_factor
new_height = raster.rio.height * upscale_factor


for year in range(1992,2023):
    year_dir_path = '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/dap.ceda.ac.uk/neodc/esacci/soil_moisture/data/daily_files/ACTIVE/v08.1/' + str(year) + '/'
    raster_path = year_dir_path + 'ESA_SM_' + str(year) + '.tif'

    raster_file = riox.open_rasterio(raster_path)
    raster_file = raster_file.rio.write_crs('EPSG:4326')
    
    raster_file_resampled = raster_file.rio.reproject(raster.rio.crs, shape=(int(new_height), int(new_width)), resampling=Resampling.bilinear)
    raster_file_resampled.rio.to_raster(raster_path)


In [29]:
for year in range(1992,2023):
    year_dir_path = '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/dap.ceda.ac.uk/neodc/esacci/soil_moisture/data/daily_files/ACTIVE/v08.1/' + str(year) + '/'
    raster_path = year_dir_path + 'ESA_SM_' + str(year) + '.tif'
    layer_polygon = catchment_file
    print('Starting on year: ' + str(year))
    dsYear = ds.sel(time = slice(str(year) + '-01-01', str(year) + '-12-31'))
    raster_file = rio.open(raster_path)
    #print(raster_file.count)
    affine = raster_file.transform
    ESA_SM = []
    for i, dt in enumerate(dsYear.time):

        array = np.array(raster_file.read(i+1)).astype(float)
        array[array == -9999] = np.nan
        zstats = rasterstats.zonal_stats(layer_polygon, array, affine=affine)
        zstats = pd.DataFrame(zstats)
        
        ESA_SM.append(zstats['mean'])
    
    dsYear['ESA_SM'] = (['time','hru'],ESA_SM)
    netcdf_path = '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/3_daily_correlation/'
    netcdf_name = 'ESA_SM_dailyCorrelation_year_' + str(year) + '.nc'
    dsYear.to_netcdf(netcdf_path + netcdf_name)

Starting on year: 1992
Starting on year: 1993
Starting on year: 1994
Starting on year: 1995
Starting on year: 1996
Starting on year: 1997
Starting on year: 1998
Starting on year: 1999
Starting on year: 2000
Starting on year: 2001
Starting on year: 2002
Starting on year: 2003
Starting on year: 2004
Starting on year: 2005
Starting on year: 2006
Starting on year: 2007
Starting on year: 2008
Starting on year: 2009
Starting on year: 2010
Starting on year: 2011
Starting on year: 2012
Starting on year: 2013
Starting on year: 2014
Starting on year: 2015
Starting on year: 2016
Starting on year: 2017
Starting on year: 2018
Starting on year: 2019
Starting on year: 2020
Starting on year: 2021
Starting on year: 2022


In [31]:
#clean up the reprojected rasters
for year in range(1992,2023):
    year_dir_path = '/Users/darrieythorsson/compHydro/data/CWARHM_data/domain_Yukon/observations/RS_Soil_moisture/ESA_SM/dap.ceda.ac.uk/neodc/esacci/soil_moisture/data/daily_files/ACTIVE/v08.1/' + str(year) + '/'
    raster_path = year_dir_path + 'ESA_SM_' + str(year) + '.tif'
    os.remove(raster_path)

In [ ]:
dsYear = xr.open_dataset(netcdf_path + netcdf_name)
dsYear

In [ ]:
raster_file = rio.open(total_path)
raster = raster_file.read(1)
raster

array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
      dtype=float32)

In [ ]:
affine = raster_file.transform
raster = raster_file.read(4)
raster[raster==-9999] = np.nan
raster_zstats = rasterstats.zonal_stats(layer_polygon, raster, affine=affine)
raster_zstats = pd.DataFrame(raster_zstats)
raster_zstats['count'].sum()

0

In [ ]:

ESA_SM = []

for i, dt in enumerate(dsSM.time):

    array = np.array(raster_file.read(i+1)).astype(float)
    array[array == -9999] = np.nan
    zstats = rasterstats.zonal_stats(layer_polygon, array, affine=affine)
    zstats = pd.DataFrame(zstats)

    ESA_SM.append(zstats['mean'])



KeyboardInterrupt: 

In [ ]:
np.array(ESA_SM)

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,   14,    6,
          0,    0,    0,    0,    0,    0,    3,    0,  291,    0,    0,
          0,  151,    0,    0,    0,    0,    0,   

AttributeError: 'numpy.int64' object has no attribute 'isna'

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [ ]:
# Set the log path and file name
logPath = intersect_path
log_suffix = '_catchment_dem_intersect_log.txt'

In [ ]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [ ]:
# Copy this script
thisFile = '1_find_HRU_elevation.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

In [ ]:
# Get current date and time
now = datetime.now()

In [ ]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Found mean HRU elevation from MERIT Hydro adjusted elevation DEM.']
    for txt in lines:
        file.write(txt)  